In [0]:
inputfile = "s3://kadentranbigdata/assignment3/twitter_combined.txt"
df = spark.read.options(header="False")\
  .option("delimiter", " ")\
  .csv(inputfile)
df.show(10)

+---------+---------+
|      _c0|      _c1|
+---------+---------+
|214328887| 34428380|
| 17116707| 28465635|
|380580781| 18996905|
|221036078|153460275|
|107830991| 17868918|
|151338729|222261763|
| 19705747| 34428380|
|222261763| 88323281|
| 19933035|149538028|
|158419434| 17434613|
+---------+---------+
only showing top 10 rows



In [0]:
userVertices = df.select("_c1").toDF("id").distinct()
userVertices.show(10)
userVertices.count()

+---------+
|       id|
+---------+
|451493376|
| 25835874|
| 24591819|
|117017147|
| 39055030|
|  3956091|
|  6100262|
|121258930|
|157165662|
|373123978|
+---------+
only showing top 10 rows

Out[21]: 81305

In [0]:
followEdges = df.select("_c0","_c1").toDF("src","dst")
followEdges.count()
followEdges.show()

+---------+---------+
|      src|      dst|
+---------+---------+
|214328887| 34428380|
| 17116707| 28465635|
|380580781| 18996905|
|221036078|153460275|
|107830991| 17868918|
|151338729|222261763|
| 19705747| 34428380|
|222261763| 88323281|
| 19933035|149538028|
|158419434| 17434613|
|149538028|153226312|
|364971269|153226312|
|100581193|279787626|
|113058991| 69592091|
|151338729|187773078|
|406628822|262802533|
|460282402| 88323281|
|280935165|437804658|
|222261763| 27633075|
|285312927|151338729|
+---------+---------+
only showing top 20 rows



In [0]:
#from graphframes import GraphFrame
from graphframes import *
followGraph = GraphFrame(userVertices,followEdges)
followGraph.cache()

Out[23]: GraphFrame(v:[id: string], e:[src: string, dst: string])

In [0]:
followGraph.vertices.show()
followGraph.edges.show()

+---------+
|       id|
+---------+
|451493376|
| 25835874|
| 24591819|
|117017147|
| 39055030|
|  3956091|
|  6100262|
|121258930|
|157165662|
|373123978|
| 18554818|
|107895465|
| 44695563|
|114596637|
|119897041|
| 16817883|
|176686379|
| 98952413|
| 15164565|
| 14363438|
+---------+
only showing top 20 rows

+---------+---------+
|      src|      dst|
+---------+---------+
|214328887| 34428380|
| 17116707| 28465635|
|380580781| 18996905|
|221036078|153460275|
|107830991| 17868918|
|151338729|222261763|
| 19705747| 34428380|
|222261763| 88323281|
| 19933035|149538028|
|158419434| 17434613|
|149538028|153226312|
|364971269|153226312|
|100581193|279787626|
|113058991| 69592091|
|151338729|187773078|
|406628822|262802533|
|460282402| 88323281|
|280935165|437804658|
|222261763| 27633075|
|285312927|151338729|
+---------+---------+
only showing top 20 rows



In [0]:
#Check the number of edges of each vertex
followGraph.degrees.show()

+---------+------+
|       id|degree|
+---------+------+
|451493376|    21|
| 25835874|     5|
| 24591819|    51|
|117017147|   125|
| 39055030|     9|
|  6100262|    21|
|  3956091|    23|
|121258930|  1477|
| 15179353|     2|
|157165662|    18|
|373123978|   160|
| 18554818|     4|
|107895465|     2|
| 44695563|    23|
|114596637|     9|
|119897041|   775|
|176686379|    35|
| 16817883|   610|
| 98952413|    13|
| 15164565|   252|
+---------+------+
only showing top 20 rows



In [0]:
access_key = ""

SECRET_KEY = ""

encoded_secret_key = SECRET_KEY.replace("/", "%2F")

aws_bucket_name = "kadentranbigdata"

mount_name = "ktbigdataassignment3part1"

try:
  dbutils.fs.mount("s3a://%s:%s@%s" % (access_key, encoded_secret_key, aws_bucket_name), "/mnt/%s" % mount_name)
except Exception as e:
  print("Directory already mounted")

Directory already mounted


In [0]:
#argument 2: location of output file
outputfile = "dbfs:/mnt/ktbigdataassignment3part1/assignment3part2Output.txt"

In [0]:
# a. Find the top 5 nodes with the highest outdegree and find the count of the number of outgoing edges in each
from pyspark.sql.functions import desc
outDeg = followGraph.outDegrees
#outDeg.orderBy(desc("outDegree")).show(5, truncate = False)

In [0]:
#Output part a
outstring = "a. Find the top 5 nodes with the highest outdegree and find the count of the number of outgoing edges in each \n"
outa = outDeg.orderBy(desc("outDegree")).head(5)
for row in outa:
  outstring = outstring + str(row) + "\n"
print(outstring)
#outa.coalesce(1).write.format("csv").mode("append").save(outputfile)

a. Find the top 5 nodes with the highest outdegree and find the count of the number of outgoing edges in each 
Row(id='3359851', outDegree=3373)
Row(id='59804598', outDegree=2467)
Row(id='7860742', outDegree=2458)
Row(id='18776017', outDegree=2272)
Row(id='5442012', outDegree=2204)



In [0]:
#b. Find the top 5 nodes with the highest indegree and find the count of the number of incoming edges in each
inDeg = followGraph.inDegrees
#inDeg.orderBy(desc("inDegree")).show(5, truncate = False)

In [0]:
#Output part b
outstring = outstring + "\n" + "b. Find the top 5 nodes with the highest indegree and find the count of the number of incoming edges in each \n"
outb = inDeg.orderBy(desc("inDegree")).head(5)
for row in outb:
  outstring = outstring + str(row) + "\n"
print(outstring)

a. Find the top 5 nodes with the highest outdegree and find the count of the number of outgoing edges in each 
Row(id='3359851', outDegree=3373)
Row(id='59804598', outDegree=2467)
Row(id='7860742', outDegree=2458)
Row(id='18776017', outDegree=2272)
Row(id='5442012', outDegree=2204)

b. Find the top 5 nodes with the highest indegree and find the count of the number of incoming edges in each 
Row(id='40981798', inDegree=8660)
Row(id='43003845', inDegree=7700)
Row(id='22462180', inDegree=7623)
Row(id='34428380', inDegree=7558)
Row(id='115485051', inDegree=4798)



In [0]:
# c. Calculate PageRank for each of the nodes and output the top 5 nodes with the highest PageRank values.
ranks = followGraph.pageRank(resetProbability=0.15, maxIter=5)
#ranks.vertices.orderBy(desc("pagerank")).select("id", "pagerank").show(5)

In [0]:
#Output part c
outstring = outstring + "\n" + "c. Calculate PageRank for each of the nodes and output the top 5 nodes with the highest PageRank values. \n"
outc = ranks.vertices.orderBy(desc("pagerank")).select("id", "pagerank").head(5)
for row in outc:
  outstring = outstring + str(row) + "\n"
print(outstring)

a. Find the top 5 nodes with the highest outdegree and find the count of the number of outgoing edges in each 
Row(id='3359851', outDegree=3373)
Row(id='59804598', outDegree=2467)
Row(id='7860742', outDegree=2458)
Row(id='18776017', outDegree=2272)
Row(id='5442012', outDegree=2204)

b. Find the top 5 nodes with the highest indegree and find the count of the number of incoming edges in each 
Row(id='40981798', inDegree=8660)
Row(id='43003845', inDegree=7700)
Row(id='22462180', inDegree=7623)
Row(id='34428380', inDegree=7558)
Row(id='115485051', inDegree=4798)

c. Calculate PageRank for each of the nodes and output the top 5 nodes with the highest PageRank values. 
Row(id='115485051', pagerank=698.6200671835986)
Row(id='116485573', pagerank=449.19995627632585)
Row(id='813286', pagerank=221.10014084582502)
Row(id='40981798', pagerank=208.75524310911425)
Row(id='7861312', pagerank=177.17293874296615)



In [0]:
# d. Run the connected components algorithm on it and find the top 5 components with the largest number of nodes.

spark.sparkContext.setCheckpointDir("/tmp/checkpoints/new")
ccGraph = GraphFrame(userVertices,followEdges)
cc = ccGraph.connectedComponents()
#cc = followGraph.connectedComponents()


In [0]:
#Output part d
outstring = outstring + "\n" + "d. Run the connected components algorithm on it and find the top 5 components with the largest number of nodes.. \n"
outd = cc.where("component != 0").groupby("component").count().orderBy(desc("count")).head(5)
for row in outd:
  outstring = outstring + str(row) + "\n"
#print(outstring)

In [0]:
#e. Run the triangle counts algorithm on each of the vertices and output the top 5 vertices with the
# largest triangle count. In case of ties, you can randomly select the top 5 vertices.
user = df.select("_c0").toDF("id").distinct()
triGraph = GraphFrame(user,followEdges)
triCount = triGraph.triangleCount()



In [0]:
#triCount.orderBy(desc("count")).show()

In [0]:
oute = triCount.select("id","count").orderBy(desc("count")).head(5)

In [0]:
#display(triCount)

In [0]:
#Output part e
outstring = outstring + "\n" + "e. Run the triangle counts algorithm on each of the vertices and output the top 5 vertices with the largest triangle count. In case of ties, you can randomly select the top 5 vertices. \n"
#oute = triCount.select("id","count").orderBy(desc("count")).head(5)
for row in oute:
  outstring = outstring + str(row) + "\n"
#print(outstring)

In [0]:
try:
  dbutils.fs.put(outputfile, outstring)
except Exception as e:
  print("File already exists")


Wrote 1408 bytes.
